In [1]:
import pandas as pd
import gspread
import os
from IPython.display import display
import locale
from datetime import datetime, date, time
# (Presume que google-auth, etc., estão instalados)

# (Presume que a autenticação (credentials2.json) está configurada 
#  para uma conta de serviço)

# Tenta configurar o locale para Português do Brasil para formatar datas
try:
    locale.setlocale(locale.LC_TIME, 'pt_BR.UTF-8')
except locale.Error:
    try:
        locale.setlocale(locale.LC_TIME, 'Portuguese_Brazil.1252')
    except locale.Error:
        print("Aviso: Não foi possível definir o locale para 'pt_BR'.")

In [2]:
import pandas as pd
import locale
from datetime import datetime, date, time
from IPython.display import display

# --- Mapa de Meses (Constante) ---
# Usamos strings '01', '02', etc., para a substituição (replace)
MESES_MAP_REPLACE = {
    'jan': '01', 'fev': '02', 'mar': '03', 'abr': '04', 
    'mai': '05', 'jun': '06', 'jul': '07', 'ago': '08', 
    'set': '09', 'out': '10', 'nov': '11', 'dez': '12'
}

def tratar_dataframe_consolidado(df_consolidado):
    """
    Limpa, formata, atualiza o status e filtra o DataFrame consolidado.
    
    1. Remove linhas onde a coluna 'Início' está vazia.
    2. (NOVO) Remove linhas onde 'Status' contém apenas números.
    3. Converte 'Início' e 'Fim' (do formato 'DD-Mês.YY-Dia') 
       usando a lógica rsplit -> replace.
    4. Adiciona 15h ao 'Início' e 11h ao 'Fim'.
    5. (NOVO) Renomeia 'CANCELADO COM CRÉDITO' para 'Cancelado'.
    6. (NOVO) Define o Status como 'Concluído' se a data 'Fim' já passou.
    7. Reformata para a string 'DD/MM/AAAA HH:MM'.
    8. Filtra o DataFrame final para manter apenas as 19 colunas especificadas.

    Args:
        df_consolidado (pd.DataFrame): O DataFrame lido da planilha 
                                     (ex: '20-jun.25-sex.').

    Returns:
        pd.DataFrame: O DataFrame tratado e filtrado.
    """
    
    print(f"Iniciando tratamento do DataFrame (Tamanho original: {len(df_consolidado)} linhas)...")
    
    # 1. Faz uma cópia para evitar modificar o original
    df_tratado = df_consolidado.copy()
    
    # --- 2. Remover linhas com 'Início' vazio ---
    linhas_antes = len(df_tratado)
    df_tratado = df_tratado[pd.notna(df_tratado['Início'])]
    df_tratado = df_tratado[df_tratado['Início'].astype(str).str.strip() != '']
    linhas_depois = len(df_tratado)
    print(f"  Removidas {linhas_antes - linhas_depois} linhas com 'Início' vazio.")

    # --- (NOVA ETAPA 1) Remover Status numérico ---
    if 'Status' in df_tratado.columns:
        linhas_antes = len(df_tratado)
        # pd.to_numeric(..., errors='coerce') cria NaN para não-números (ex: 'Cancelado')
        # .isna() marca True para não-números (que queremos manter)
        linhas_para_manter = pd.to_numeric(df_tratado['Status'], errors='coerce').isna()
        df_tratado = df_tratado[linhas_para_manter]
        print(f"  Removidas {linhas_antes - len(df_tratado)} linhas com 'Status' puramente numérico.")
    else:
        print("  Aviso: Coluna 'Status' não encontrada para filtro numérico.")

    # --- 3. Parsear e Formatar 'Início' e 'Fim' (Cria _dt) ---
    print("  Parseando e formatando datas (Início e Fim)...")
    colunas_para_processar = {'Início': '15 hours', 'Fim': '11 hours'}
    colunas_dt_temporarias = [] 

    for col_nome, timedelta_str in colunas_para_processar.items():
        if col_nome not in df_tratado.columns:
            print(f"  Aviso: Coluna '{col_nome}' não encontrada. Pulando.")
            continue
        col_dt_temp = f"{col_nome}_dt"
        colunas_dt_temporarias.append(col_dt_temp)
        # Etapas A, B, C, D (Lógica de parse e adição de horas)
        datas_limpas = df_tratado[col_nome].astype(str).str.rsplit('-', n=1).str[0].str.strip()
        temp_series = datas_limpas.str.lower()
        for pt_mes, num_mes in MESES_MAP_REPLACE.items():
            temp_series = temp_series.str.replace(pt_mes, num_mes)
        df_tratado[col_dt_temp] = pd.to_datetime(temp_series, format='%d-%m.%y', errors='coerce')
        timedelta = pd.to_timedelta(timedelta_str)
        valid_dates = df_tratado[col_dt_temp].notna()
        df_tratado.loc[valid_dates, col_dt_temp] = df_tratado.loc[valid_dates, col_dt_temp] + timedelta

    # --- 4. Remover linhas onde o parse de data falhou (virou NaT) ---
    if colunas_dt_temporarias:
        linhas_antes_parse = len(df_tratado)
        df_tratado.dropna(subset=colunas_dt_temporarias, inplace=True)
        erros_parse = linhas_antes_parse - len(df_tratado)
        if erros_parse > 0:
            print(f"  Removidas {erros_parse} linhas onde 'Início' ou 'Fim' tinham formato de data inválido.")
            
    # --- 5. (NOVAS ETAPAS 2 & 3) Atualização de Status ---
    if 'Status' in df_tratado.columns:
        
        # (ETAPA 2) Renomear 'CANCELADO COM CRÉDITO' para 'Cancelado'
        print("  Renomeando Status 'CANCELADO COM CRÉDITO'...")
        df_tratado['Status'] = df_tratado['Status'].replace(
            'CANCELADO COM CRÉDITO', 'Cancelado'
        )
        
        # (ETAPA 3) Colocar 'Concluído' se 'Fim_dt' já passou
        if 'Fim_dt' in df_tratado.columns:
            print("  Atualizando Status para 'Concluído' em reservas passadas...")
            # Pega a data/hora exata de agora
            agora = pd.to_datetime('now')
            # Define a condição (Onde Fim_dt é anterior a agora)
            condicao_concluido = df_tratado['Fim_dt'] < agora
            # Atualiza 'Status' usando .loc
            df_tratado.loc[condicao_concluido, 'Status'] = 'Concluído'
            print(f"    {condicao_concluido.sum()} linhas atualizadas para 'Concluído'.")
        else:
            print("  Aviso: Coluna 'Fim_dt' não encontrada, não foi possível atualizar para 'Concluído'.")
            
    else:
        print("  Aviso: Coluna 'Status' não encontrada para aplicar regras de negócio.")

    # --- 6. Reformatar colunas originais para 'DD/MM/AAAA HH:MM' ---
    print("  Reformatando colunas de data para 'DD/MM/YYYY HH:MM'...")
    if 'Início_dt' in df_tratado.columns:
        df_tratado['Início'] = df_tratado['Início_dt'].dt.strftime('%d/%m/%Y %H:%M')
    if 'Fim_dt' in df_tratado.columns:
        df_tratado['Fim'] = df_tratado['Fim_dt'].dt.strftime('%d/%m/%Y %H:%M')
    
    # --- 7. Limpar colunas temporárias ---
    df_tratado = df_tratado.drop(columns=colunas_dt_temporarias, errors='ignore')

    # --- 8. Selecionar e Reordenar Colunas Finais ---
    print("  Filtrando para colunas finais desejadas...")
    colunas_desejadas = [
        'idReserva', 'Apartamento', 'Início', 'Fim', 'Dias', 'Pessoas',
        'Quem', 'Origem', 'Recebido', 'Despesas', 'A receber', 'Total Lq',
        'Total BT', 'Diária BT', 'Diária Lq', 'Com.', 'Diária BT PAX',
        'Diária Lq PAX', 'Status','Data Reserva'
    ]

    colunas_finais_presentes = [col for col in colunas_desejadas if col in df_tratado.columns]
    colunas_perdidas = [col for col in colunas_desejadas if col not in df_tratado.columns]
    if colunas_perdidas:
        print(f"  Aviso: As seguintes colunas não foram encontradas e não estarão no resultado: {colunas_perdidas}")

    df_final_filtrado = df_tratado[colunas_finais_presentes]
    
    print(f"Tratamento concluído. (Tamanho final: {len(df_final_filtrado)} linhas)")
    
    return df_final_filtrado

In [3]:
def ler_abas_planilha(sheet_key, lista_abas, credentials_path):
    """
    Autentica no Google Sheets, abre uma planilha específica e lê
    múltiplas abas (planilhas/worksheets) para DataFrames Pandas.
    
    Assume que os cabeçalhos estão na Linha 3 (índice 2) e os 
    dados começam na Linha 4 (índice 3) para TODAS as abas.

    Args:
        sheet_key (str): A chave (ID) da Planilha Google.
        lista_abas (list): Uma lista de strings com os nomes das abas
                           que devem ser lidas (ex: ['SM-C108', 'SM-D014']).
        credentials_path (str): Caminho para o arquivo JSON da conta de serviço.

    Returns:
        dict: Um dicionário onde as chaves são os nomes das abas e 
              os valores são os DataFrames Pandas correspondentes.
              Retorna None para abas que falharam na leitura.
    """
    
    print(f"\n--- Iniciando leitura da Planilha Google (Key: {sheet_key}) ---")
    
    # Dicionário para armazenar os DataFrames resultantes
    dataframes_lidos = {}

    # Verifica se o arquivo de credenciais existe
    if not os.path.exists(credentials_path):
        print(f"ERRO: Arquivo de credenciais '{credentials_path}' não encontrado.")
        return None # Retorna None se a autenticação falhar

    try:
        # 1. Autenticar e Abrir a Planilha (uma única vez)
        print("Autenticando com conta de serviço...")
        gc = gspread.service_account(filename=credentials_path)
        print("Autenticação OK.")

        print(f"Abrindo planilha...")
        sh = gc.open_by_key(sheet_key)
        print("Planilha aberta com sucesso.")

    except gspread.exceptions.APIError as e_api:
        print(f"ERRO de API do Google: {e_api}")
        print("Verifique permissões, quotas e se a API está habilitada.")
        return None
    except gspread.exceptions.SpreadsheetNotFound:
         print(f"ERRO: Planilha com key '{sheet_key}' não encontrada ou sem permissão.")
         return None
    except Exception as e_auth:
        print(f"ERRO inesperado durante a autenticação ou abertura: {e_auth}")
        return None

    # 2. Iterar sobre a lista de abas fornecida
    for nome_aba in lista_abas:
        print(f"\n  --- Processando aba: '{nome_aba}' ---")
        try:
            # Seleciona a Aba
            worksheet = sh.worksheet(nome_aba)
            
            # Lê Todos os Valores
            # value_render_option='FORMATTED_VALUE' lê como exibido na planilha
            all_values = worksheet.get_all_values(value_render_option='FORMATTED_VALUE')
            
            # Processa com Pandas (Cabeçalho na Linha 3, Dados da Linha 4)
            if len(all_values) < 4:
                print(f"    Aviso: Aba '{nome_aba}' tem menos de 4 linhas. "
                      "Retornando DataFrame vazio.")
                # Tenta pegar o cabeçalho se existir
                headers = all_values[2] if len(all_values) >= 3 else []
                dataframes_lidos[nome_aba] = pd.DataFrame(columns=headers)
                continue # Pula para a próxima aba

            headers = all_values[2] # Linha 3 (índice 2)
            data = all_values[3:]   # Linha 4 (índice 3) em diante

            # Cria o DataFrame
            df = pd.DataFrame(data, columns=headers)
            print(f"    Aba '{nome_aba}' lida com sucesso. {len(df)} linhas de dados encontradas.")
            dataframes_lidos[nome_aba] = df

        except gspread.exceptions.WorksheetNotFound:
            print(f"    ERRO: Aba '{nome_aba}' não foi encontrada na planilha. Pulando.")
            dataframes_lidos[nome_aba] = None # Indica falha para esta aba
        except Exception as e:
            print(f"    ERRO inesperado ao processar a aba '{nome_aba}': {e}")
            dataframes_lidos[nome_aba] = None # Indica falha para esta aba

    print("\n--- Leitura de todas as abas concluída. ---")
    return dataframes_lidos

In [4]:
def unir_dataframes_lidos(dfs_lidos):
    """
    Recebe um dicionário de DataFrames (lidos das abas), concatena todos,
    substitui a coluna 'Apartamento' pelo nome da aba (chave do dicionário),
    e cria uma nova coluna 'idReserva' sequencial.
    
    Esta versão inclui uma correção para colunas duplicadas nas abas de origem,
    que é a causa provável do InvalidIndexError.

    Args:
        dfs_lidos (dict): Dicionário no formato {nome_aba: pd.DataFrame}.

    Returns:
        pd.DataFrame: Um único DataFrame consolidado e transformado,
                      ou um DataFrame vazio se houver erro.
    """
    
    lista_dfs_processados = []

    if not dfs_lidos:
        print("Dicionário de DataFrames está vazio.")
        return pd.DataFrame()

    print("Iniciando união e transformação dos DataFrames...")

    # Itera sobre o dicionário (nome_aba = chave, df_aba = valor)
    for nome_aba, df_aba in dfs_lidos.items():
        
        # Pula abas que falharam na leitura (None) ou estão vazias
        if df_aba is None or df_aba.empty:
            print(f"  Aviso: Aba '{nome_aba}' está vazia ou teve erro na leitura. Pulando.")
            continue
            
        try:
            df_processado = df_aba.copy()
            
            # --- CORREÇÃO OBRIGATÓRIA PARA O ERRO InvalidIndexError ---
            # Verifica se os nomes das colunas nesta aba são únicos
            if not df_processado.columns.is_unique:
                print(f"  Aviso: Corrigindo nomes de colunas duplicados na aba '{nome_aba}'.")
                cols = pd.Series(df_processado.columns)
                for dup in cols[cols.duplicated()].unique():
                    # Pega todas as posições da coluna duplicada
                    dup_indices = cols[cols == dup].index.tolist()
                    # Renomeia a partir da *segunda* ocorrência
                    for i, idx in enumerate(dup_indices[1:], start=1):
                        cols[idx] = f"{dup}.{i}"
                df_processado.columns = cols
            # --- FIM DA CORREÇÃO ---

            # 2. Substitui a coluna 'Apartamento' pelo nome da aba
            df_processado['Apartamento'] = nome_aba
            
            # 3. Descarta a coluna 'Id' original, se existir
            if 'Id' in df_processado.columns:
                df_processado = df_processado.drop(columns=['Id'])
                
            lista_dfs_processados.append(df_processado)
            
        except Exception as e:
            print(f"  ERRO ao processar a aba '{nome_aba}': {e}. Pulando.")

    # Verifica se algum DataFrame foi processado
    if not lista_dfs_processados:
        print("Nenhum DataFrame válido foi processado.")
        return pd.DataFrame()

    # 4. Concatena (empilha) todos os DataFrames processados em um só
    # Esta linha agora deve funcionar graças à correção de colunas duplicadas
    df_consolidado = pd.concat(lista_dfs_processados, ignore_index=True, sort=False)
    
    # 5. Adiciona o novo ID sequencial (chave primária)
    df_consolidado['idReserva'] = df_consolidado.index + 1 # Começa em 1
    
    # 6. Reordena as colunas (opcional, mas bom para visualização)
    colunas_principais = ['idReserva', 'Apartamento']
    colunas_restantes = [col for col in df_consolidado.columns if col not in colunas_principais]
    df_consolidado = df_consolidado[colunas_principais + colunas_restantes]
    
    print(f"União concluída. DataFrame final com {len(df_consolidado)} linhas.")
    
    return df_consolidado

In [5]:
import gspread
import pandas as pd
import os
# (Certifique-se de que as bibliotecas google-auth, etc., estão instaladas)

def salvar_df_no_gsheet(df, sheet_key, worksheet_name, credentials_path, clear_sheet=True):
    """
    Salva um DataFrame do Pandas em uma aba específica de uma Planilha Google.
    Cria a aba, ajusta colunas, aplica um filtro para 'Status' em branco
    E ORDENA pela coluna 'Início'.
    """
    print(f"\n--- Iniciando Upload para Google Sheet ---")
    print(f"  Planilha Key: {sheet_key}")
    print(f"  Aba         : {worksheet_name}")

    if not os.path.exists(credentials_path):
        print(f"ERRO: Arquivo de credenciais '{credentials_path}' não encontrado.")
        return False

    try:
        # 1. Autenticar
        print("Autenticando com conta de serviço...")
        gc = gspread.service_account(filename=credentials_path)
        print("Autenticação OK.")

        # 2. Abrir a Planilha
        print(f"Abrindo planilha...")
        sh = gc.open_by_key(sheet_key)
        print("Planilha aberta.")

        # 3. Selecionar ou Criar a Aba
        try:
            worksheet = sh.worksheet(worksheet_name)
            print(f"Aba '{worksheet_name}' encontrada.")
        except gspread.exceptions.WorksheetNotFound:
            print(f"Aba '{worksheet_name}' não encontrada. Criando nova aba...")
            worksheet = sh.add_worksheet(title=worksheet_name, 
                                         rows=str(len(df) + 10), 
                                         cols=str(len(df.columns) + 2))
            print(f"Aba '{worksheet_name}' criada.")

        # 4. Limpar a Aba (se solicitado)
        if clear_sheet:
            print(f"Limpando conteúdo existente da aba '{worksheet_name}'...")
            worksheet.clear()
            print("Aba limpa.")

        # 5. Preparar os Dados (Cabeçalho + Linhas)
        print("Preparando dados para upload...")
        df_upload = df.fillna('').astype(str)
        dados_preparados = [df_upload.columns.values.tolist()] + df_upload.values.tolist()
        num_rows, num_cols = df_upload.shape
        print(f"Dados preparados: {num_rows + 1} linhas, {num_cols} colunas.")

        # 6. Atualizar a Aba
        print(f"Enviando dados para a aba '{worksheet_name}' (iniciando em A1)...")
        
        # --- CORREÇÃO APLICADA AQUI (usando argumentos nomeados) ---
        worksheet.update(
            range_name='A1', 
            values=dados_preparados, 
            value_input_option='USER_ENTERED'
        )
        # -----------------------------------------------------------
        
        print("Dados enviados com sucesso!")

        # 7. Ajustar o Tamanho das Colunas
        print("Ajustando o tamanho das colunas...")
        try:
            sheet_id = worksheet.id
            body_resize = {
                "requests": [
                    {
                        "autoResizeDimensions": {
                            "dimensions": {
                                "sheetId": sheet_id,
                                "dimension": "COLUMNS",
                                "startIndex": 0,
                                "endIndex": num_cols
                            }
                        }
                    }
                ]
            }
            sh.batch_update(body_resize)
            print("Tamanho das colunas ajustado com sucesso.")
        except Exception as e_resize:
            print(f"  AVISO: Falha ao tentar ajustar o tamanho das colunas: {e_resize}")
            
        # (ETAPA ATUALIZADA) ADICIONAR FILTRO PRÉ-DEFINIDO E ORDENAÇÃO
        print("Adicionando filtro (Status=Em Branco) e ordenando por 'Início'...")
        try:
            # Precisamos encontrar os índices (base 0) das colunas
            try:
                status_col_index = df_upload.columns.get_loc('Status') 
                inicio_col_index = df_upload.columns.get_loc('Início')
            except KeyError as e:
                print(f"  ERRO: Coluna 'Status' ou 'Início' não encontrada no DataFrame. Não é possível aplicar filtro/ordenação. Erro: {e}")
                return True 

            sheet_id = worksheet.id
            
            body_filter_sort = {
                "requests": [
                    {
                        "setBasicFilter": {
                            "filter": {
                                "range": {
                                    "sheetId": sheet_id,
                                    "startRowIndex": 0, # Linha 1 (cabeçalho)
                                    "endRowIndex": num_rows + 1,
                                    "startColumnIndex": 0,
                                    "endColumnIndex": num_cols
                                },
                                "sortSpecs": [
                                    {
                                        "dimensionIndex": inicio_col_index, # Índice da coluna 'Início'
                                        "sortOrder": "ASCENDING"
                                    }
                                ],
                                "criteria": {
                                    str(status_col_index): {
                                        "condition": {
                                            "type": "BLANK"
                                        }
                                    }
                                }
                            }
                        }
                    }
                ]
            }
            sh.batch_update(body_filter_sort)
            print("Autofiltro (Status=Branco) e ordenação por 'Início' aplicados com sucesso.")

        except Exception as e_filter:
            print(f"  AVISO: Falha ao tentar adicionar o autofiltro/ordenação: {e_filter}")
            
        return True 

    except gspread.exceptions.APIError as e_api:
        print(f"ERRO de API do Google: {e_api}")
        print("Verifique permissões, quotas e se a API está habilitada.")
        return False
    except gspread.exceptions.SpreadsheetNotFound:
         print(f"ERRO: Planilha com key '{sheet_key}' não encontrada ou sem permissão.")
         return False
    except Exception as e:
        print(f"ERRO inesperado durante operação com Google Sheets: {e}")
        return False

In [6]:
# --- Configuração ---
# Chave da sua planilha (extraída da URL)
SHEET_KEY = '1FqgTQAGebxvHUdVXI471HpAaXeXyCFdFWur7Pck0hLY'
# Arquivo de credenciais JSON da sua conta de serviço
CREDS_FILE = 'credentials2.json' 
# O nome exato da aba onde os dados consolidados serão salvos
WORKSHEET_NAME = 'Reservas Consolidadas'

# A lista de abas que você quer ler
abas_para_ler = [
    'SM-C108',
    'SM-D014',
    'CBL004',
    'AP-101',
    'AP-201'
]

# --- Execução ---
# Chama a função para ler todas as abas
dfs_lidos = ler_abas_planilha(SHEET_KEY, abas_para_ler, CREDS_FILE)
df_consolidado_final = unir_dataframes_lidos(dfs_lidos)
# Chama a nova função de tratamento
df_tratado = tratar_dataframe_consolidado(df_consolidado_final)
display(df_tratado.head())

# --- Verificação ---
# Verifica se o DataFrame 'df_consolidado_final' (da sua célula anterior) existe
if isinstance(df_consolidado_final, pd.DataFrame):
    
    if not df_tratado.empty:
        print(f"Iniciando salvamento do 'df_tratado' ({len(df_tratado)} linhas)...")
        
        # --- Chamada da Função ---
        sucesso_upload = salvar_df_no_gsheet(
            df=df_tratado,
            sheet_key=SHEET_KEY,
            worksheet_name=WORKSHEET_NAME,
            credentials_path=CREDS_FILE,
            clear_sheet=True  # Garante que a aba será limpa antes de salvar
        )

        if sucesso_upload:
            print("\nUpload do DataFrame consolidado concluído com sucesso!")
        else:
            print("\nFalha no upload do DataFrame consolidado. Verifique os logs de erro.")
    else:
        print("AVISO: 'df_consolidado_final' está vazio. Nada para salvar.")

else:
    print("ERRO: A variável 'df_consolidado_final' não foi encontrada ou não é um DataFrame.")
    print("Por favor, execute a célula que define 'df_consolidado_final' (função 'unir_dataframes_lidos') primeiro.")




--- Iniciando leitura da Planilha Google (Key: 1FqgTQAGebxvHUdVXI471HpAaXeXyCFdFWur7Pck0hLY) ---
Autenticando com conta de serviço...
Autenticação OK.
Abrindo planilha...
Planilha aberta com sucesso.

  --- Processando aba: 'SM-C108' ---
    Aba 'SM-C108' lida com sucesso. 88 linhas de dados encontradas.

  --- Processando aba: 'SM-D014' ---
    Aba 'SM-D014' lida com sucesso. 15 linhas de dados encontradas.

  --- Processando aba: 'CBL004' ---
    Aba 'CBL004' lida com sucesso. 375 linhas de dados encontradas.

  --- Processando aba: 'AP-101' ---
    Aba 'AP-101' lida com sucesso. 580 linhas de dados encontradas.

  --- Processando aba: 'AP-201' ---
    Aba 'AP-201' lida com sucesso. 578 linhas de dados encontradas.

--- Leitura de todas as abas concluída. ---
Iniciando união e transformação dos DataFrames...
União concluída. DataFrame final com 1636 linhas.
Iniciando tratamento do DataFrame (Tamanho original: 1636 linhas)...
  Removidas 59 linhas com 'Início' vazio.
  Removidas 2 li

,idReserva,Apartamento,Início,Fim,Dias,Pessoas,Quem,Origem,Recebido,Despesas,A receber,Total Lq,Total BT,Diária BT,Diária Lq,Com.,Diária BT PAX,Diária Lq PAX,Status,Data Reserva
0,1,SM-C108,20/06/2025 15:00,24/06/2025 11:00,4,5,Mariana,Direto,"1.800,00",,,"1.800,00","1.800,00","450,00","450,00",0%,"90,00","90,00",Concluído,
1,2,SM-C108,08/07/2025 15:00,13/07/2025 11:00,5,5,Alex Palmer +55 63 99991 9458,Booking,"1.084,02",,"0,00","1.084,02","1.246,00","249,20","216,80",13%,"49,84","43,36",Concluído,
2,3,SM-C108,13/07/2025 15:00,15/07/2025 11:00,2,2,‪Weiman Pereira +55 66 98448-1056‬,Airbnb,,,"400,96","400,96","490,26","245,13","200,48",18%,"122,56","100,24",Concluído,
3,4,SM-C108,16/07/2025 15:00,18/07/2025 11:00,2,5,Paula Andrade +55 81 99716 3944,Booking,"436,74",,"0,00","436,74","502,00","251,00","218,37",13%,"50,20","43,67",Concluído,
4,5,SM-C108,18/07/2025 15:00,20/07/2025 11:00,2,4,Cláudia +55 81 99974-5581‬,Airbnb,"290,00",,"292,39","582,39","712,09","356,05","291,20",18%,"89,01","72,80",Concluído,


Iniciando salvamento do 'df_tratado' (1575 linhas)...

--- Iniciando Upload para Google Sheet ---
  Planilha Key: 1FqgTQAGebxvHUdVXI471HpAaXeXyCFdFWur7Pck0hLY
  Aba         : Reservas Consolidadas
Autenticando com conta de serviço...
Autenticação OK.
Abrindo planilha...
Planilha aberta.
Aba 'Reservas Consolidadas' encontrada.
Limpando conteúdo existente da aba 'Reservas Consolidadas'...
Aba limpa.
Preparando dados para upload...
Dados preparados: 1576 linhas, 20 colunas.
Enviando dados para a aba 'Reservas Consolidadas' (iniciando em A1)...
Dados enviados com sucesso!
Ajustando o tamanho das colunas...
Tamanho das colunas ajustado com sucesso.
Adicionando filtro (Status=Em Branco) e ordenando por 'Início'...
Autofiltro (Status=Branco) e ordenação por 'Início' aplicados com sucesso.

Upload do DataFrame consolidado concluído com sucesso!
